In [1]:
import os
import json
import numpy as np
import pandas as pd

In [2]:
filepath = '../results_new_multi_hop/llama_2_7b_chat_hf/art_sculpture_FT-L.json'
with open(filepath, 'r') as file:
    metrics = json.load(file)
metrics[0]

{'pre': {'edit_acc': [0],
  'edit_output': ['Art'],
  'locality': {},
  'portability': {},
  'yes_questions': {},
  'no_questions': {},
  'multiple_choice_questions': {},
  'reversed_relation_questions': {},
  'questions_2hop': {'2hop_acc': [0], '2hop_output': ['Claes Oldenburg']},
  'questions_3hop': {'3hop_acc': [0], '3hop_output': ['"1971"']},
  'questions_4hop': {'4hop_acc': [1], '4hop_output': ['Texas']},
  'questions_5hop': {'5hop_acc': [1], '5hop_output': ['Amarillo']},
  'questions_6hop': {'6hop_acc': [0], '6hop_output': ['Texas']},
  'harm_original_text': {}},
 'case_id': 0,
 'requested_edit': {'prompt': 'What does Cadillac Ranch depict?',
  'target_new': 'motor car',
  'ground_truth': '<|endoftext|>',
  'portability': {},
  'locality': {},
  'yes_questions': {},
  'no_questions': {},
  'multiple_choice_questions': {},
  'reversed_relation_questions': {},
  'questions_2hop': {'2hop': {'prompt': 'Who is the artist that created the artwork that Cadillac Ranch depicts?',
    'gro

In [3]:
filepath = '../results/llama_2_7b_chat_hf/art_sculpture_FT-L.json'
with open(filepath, 'r') as file:
    metrics = json.load(file)
metrics[0]

{'pre': {'edit_acc': [0],
  'edit_output': ['Art'],
  'locality': {'locality_output': ['Texas']},
  'portability': {},
  'yes_questions': {'yes_acc': [1], 'yes_output': ['Yes']},
  'no_questions': {'no_acc': [0], 'no_output': ['Yes']},
  'multiple_choice_questions': {'multiple_choice_acc': [1],
   'multiple_choice_output': ['D. motor car']},
  'reversed_relation_questions': {'reversed_relation_acc': [1],
   'reversed_relation_output': ['Cadillac']},
  'questions_2hop': {'2hop_acc': [0], '2hop_output': ['Claes Oldenburg']},
  'questions_3hop': {'3hop_acc': [0], '3hop_output': ['"1971"']},
  'questions_4hop': {'4hop_acc': [1], '4hop_output': ['Texas']},
  'questions_5hop': {'5hop_acc': [1], '5hop_output': ['Amarillo']},
  'questions_6hop': {'6hop_acc': [0], '6hop_output': ['Amarillo']},
  'harm_original_text': {},
  'rephrase_acc': [0],
  'rephrase_output': ['Art']},
 'case_id': 0,
 'requested_edit': {'prompt': 'What does Cadillac Ranch depict?',
  'target_new': 'motor car',
  'ground_tr

In [10]:
metrics[0]['pre'].keys()

dict_keys(['edit_acc', 'edit_output', 'locality', 'portability', 'yes_questions', 'no_questions', 'multiple_choice_questions', 'reversed_relation_questions', 'questions_2hop', 'questions_3hop', 'questions_4hop', 'questions_5hop', 'questions_6hop', 'harm_original_text', 'rephrase_acc', 'rephrase_output'])

there are 2 folders: results and results_new_multi_hop. Each folder has multile json file with same name but partially different content. Use the questions_nhop (n=2,3,4,5,6) in the json files under results_new_multi_hop to update questions_nhop in files from results folder
(can't because new multihop questions drop some rows probably due to NA, 5 csv files have less rows)

In [3]:
import shutil
from pathlib import Path

folder = '../results/hallu_edit'
# Get all json files in the folder
json_files = Path(folder).glob('**/*.json')

for json_file in json_files:
    # Read the json file
    with open(json_file, 'r') as f:
        data = json.load(f)
        
    # Remove n-hop questions from each item in the list
    for item in data:
        if 'pre' in item:
            for n in range(2,7):
                item['pre'].pop(f'questions_{n}hop', None)
        if 'post' in item:
            for n in range(2,7):
                item['post'].pop(f'questions_{n}hop', None)
    
    # Write back the modified data
    # with open(json_file, 'w') as f:
        # json.dump(data, f, indent=4)


In [11]:
for item_idx in range(len(target_data)):
    if item_idx >= len(source_data):
        print(f"Source data has fewer items than target data. Stopping at index {item_idx}")

In [8]:
def calc_metric(metric_list, percent=100, std_flag=False):
    mean_val = np.mean(metric_list)
    std_val = np.std(metric_list)
    if std_flag:
        return f"{mean_val*percent:.2f}±{std_val:.2f}"
    else:
        return np.round(mean_val*percent, 2)
    

def summarize_json_to_df(json_folder, std_flag=False):
    metrics_list = []

    for filename in os.listdir(json_folder):
        if filename.endswith('.json'):
            with open(os.path.join(json_folder, filename), 'r') as file:
                metrics = json.load(file)
                edit_method, data, model = filename.split('_')[:3]

                if edit_method == 'IKE':
                    accuracy_pre = calc_metric([e['pre']['rewrite_acc'] for e in metrics])
                    accuracy_post = calc_metric([e['post']['rewrite_acc'] for e in metrics])
                else:
                    accuracy_pre = calc_metric([e['pre']['rewrite_acc'][0] for e in metrics])
                    accuracy_post = calc_metric([e['post']['rewrite_acc'][0] for e in metrics])

                results = {
                    "data": data,
                    "edit_method": edit_method,
                    "accuracy_pre": accuracy_pre,
                    "accuracy_post": accuracy_post,
                }
                metrics_list.append(results)
    df = pd.DataFrame(metrics_list)
    df = df[df['edit_method'] != 'SERAC']
    df = df[df['data'].isin(['recent', 'zsre', 'wikibio'])]
    return df


summarize_json_to_df("../results/know_edit/eval").sort_values(by=['data', 'edit_method'])

,data,edit_method,accuracy_pre,accuracy_post
18,recent,FT,47.40,100.00
0,recent,FT-L,47.40,56.30
17,recent,IKE,47.40,99.97
4,recent,LoRA,47.40,100.00
19,recent,MEMIT,47.40,97.10
12,recent,ROME,47.45,97.37
11,wikibio,FT,61.35,100.00
26,wikibio,FT-L,61.33,66.70
3,wikibio,IKE,61.33,100.00
13,wikibio,LoRA,61.33,100.00
